In [2]:
from utils import *
import pymongo

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["db"]
birds = mydb["electric_scooters"]

In [4]:
def get_birds_ids(birds):
    return list(birds.distinct("id"))

In [5]:
birds_ids = get_birds_ids(birds)
print(len(birds_ids))

7642


In [51]:
def get_distance_and_moves(bird_id, birds, time):
    if time == "any":
        bird_by_id = birds.find({"id": bird_id})
    else:
        #bird_by_id = birds.find({"id": bird_id},{"time": "/2019/"})
        bird_by_id = birds.find({"$and": [{"id": bird_id}, {"time":{"$regex": u"" + time +""}}]})
    bird_by_id = list(bird_by_id)
    
    #print(len(bird_by_id))
    total_distance = 0
    used = 0
    print("----------------------------------------------------------")
    print("ID: " + bird_id)
    for i in range(len(bird_by_id) - 1):
        previous = bird_by_id[i]
        actual = bird_by_id[i + 1]
        #print(calculate_distance(previous.get("location"), actual.get("location"))
        actual_distance = calculate_distance(previous.get("location"), actual.get("location"))
        if(actual_distance > 0.001 and (int(previous.get("battery_level")) > int(actual.get("battery_level")))):
            print(previous.get("time") + " " + previous.get("battery_level") + " " + actual.get("battery_level") +" " + str(actual_distance))
            used += 1
            total_distance += actual_distance
    return {"distance":total_distance, "used": used}

In [41]:
test_id = birds_ids[300]
get_distance_and_moves(test_id, birds, "2019")

2019-10-28 23:28:57 26 21 0.003314663720592317
2019-10-28 23:25:08 27 26 0.09666479730498245
2019-10-28 23:27:41 26 21 0.004787932682736436


{'total_distance': 0.1047673937083112, 'used': 3}

In [85]:
all_time = "any"
days = ["2019-10-29", "2019-10-30", "2019-10-31", "2019-11-01", "2019-11-02", "2019-11-03", "2019-11-04"]

In [61]:
def get_total(bird_ids, birds, time):
    total_used = 0
    total_dist = 0
    interator = 0
    bird_ids_len = len(bird_ids)
    for bird_id in bird_ids:
        interator +=1
        dist_movs = get_distance_and_moves(bird_id, birds, time)
        print("ID number : " + str(interator) + " of " + str(bird_ids_len))
        total_used += dist_movs.get("used")
        total_dist += dist_movs.get("distance")
    return {"distance":total_dist, "used": total_used}

In [63]:
#Długi czsa wykonania
get_total(birds_ids, birds, all_time)

In [64]:
hours = [" 00:", " 01:", " 02:", " 03:", " 04:", " 05:", " 06:", " 07:", " 08:", " 09:" ," 10:", " 11:", " 12:",
        " 13:", " 14:", " 15:", " 16:", " 17:", " 18:", " 19:", " 20:", " 21:", " 22:", " 23:" ]

In [66]:
def ids_for_time(birds, time):
    if time == "any":
        return birds.distinct("id")
    else:
        return birds.find({"time":{"$regex": u"" + time +""}}).distinct("id")

In [82]:
len(ids_for_time(birds, " 03:"))

1851

In [ ]:
result = list()
for day in days:
    print("Day: " + day)
    ids = ids_for_time(birds, day)
    print("Number of ids: " + str(len(ids)))
    result.append(get_total(ids, birds, day))

Day: 2019-10-29
Number of ids: 919
----------------------------------------------------------
ID: 8b6835c4-b0f4-4509-bc25-01d932f79082
2019-10-29 00:25:24 78 75 0.001210487287143155
2019-10-29 00:26:37 78 75 0.001210487287143155
2019-10-29 00:27:53 78 75 0.001210487287143155
2019-10-29 00:29:06 78 75 0.001210487287143155
2019-10-29 00:24:08 78 75 0.0032940198374730102
2019-10-29 00:40:17 75 73 0.008256041758550959
ID number : 1 of 919
----------------------------------------------------------
ID: 9f40d804-6d65-4c59-9bba-44d9b1939c4d
ID number : 2 of 919
----------------------------------------------------------
ID: 911354b4-7b85-4b65-a7a0-5f7b86e5f79b
ID number : 3 of 919
----------------------------------------------------------
ID: 7d4eb88c-b7fa-4176-8325-9f0496112996
ID number : 4 of 919
----------------------------------------------------------
ID: 6d5a7ba3-525c-47fd-ae3b-849e04929ff3
ID number : 5 of 919
----------------------------------------------------------
ID: 54f6a734-bbc5-